In [ ]:
from __future__ import division, unicode_literals, print_function
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os.path, os, sys, json
plt.rcParams.update({'font.size': 16, 'figure.figsize': [8.0, 6.0]})
workdir=%pwd

### Download and build the Monte Carlo package _Faunus_

To simulate the interaction of rigid fibrils with a charged, planar surface we use a custom Metropolis Monte Carlo (MC) program build within the [Faunus framework](http://github.com/mlund/faunus.git). The sections below will fetch the complete faunus project and compile the appropriate version of the program.

In [ ]:
%%bash -s "$workdir"
cd $1
if [ ! -d "faunus/" ]; then
  git clone https://github.com/mlund/faunus.git
  cd faunus
  git checkout fceb673dc51244eb37fc1f26a89dd910f0228b33
  cp ../montecarlo.cpp src/examples/stripes.cpp
else
  cd faunus
fi
CXX=clang++ CC=clang cmake . -DCMAKE_BUILD_TYPE=Release &>/dev/null 
make example_stripes

### Run Metropolis Monte Carlo Simulation

This will generate an input file for the MC program, run it and print simulation details.
Finally, the positions at the end of the simulation are visualized.

In [ ]:
mcdir = workdir+'/faunus/src/examples'
%cd $mcdir
%rm -f state

N=40          # number of particles
diameter=53   # particle diameter
L=600.        # side lengths of the quadratic simulation plane

# generate input file for MC program
js = {
    "atomlist" : { "MP" : { "r":diameter*0.5, "dp":50 } },
    "moleculelist" : {
        "myparticles" : { "atoms":"MP", "atomic":True, "Ninit" : N, "insdir":"1 1 0" }
    },
    "energy" : {
        "nonbonded" : {
            "repulsionr3" : {"prefactor":0.5e6}
        }
    },
    "moves" : {
        "atomtranslate" : {
          "myparticles" : { "peratom":True, "dir":"1 1 0" }
        }
    },
    "system" : {
        "cuboid" : { "len" : L }
    }
}
with open('montecarlo.json', 'w+') as f:
    f.write(json.dumps(js, indent=4))

# if MC executable exists, run it and visualize final configuration
exe='./stripes'
if ( os.access( exe, os.X_OK )):
    !./stripes
    x,y = np.genfromtxt('confout.pqr', invalid_raise=False, usecols=(5,6), unpack=True, skip_footer=1)
    x=x+L/2.0
    y=y+L/2.0
    plt.plot(x,y, 'ro', ms=diameter*0.5)
    plt.xlabel(r'(you fill in all axis labels!)')

### Calculate histogram for uniform distribution

We now generate the distribution of particle distances for the case where these are completely uncorrelated, i.e. for an ideal 2D gas confined on a square with hard boundaries. This is done by randomly placing a large number of particles (the more, the higher accuracy) and build a histogram of distances.

In [ ]:
from scipy.spatial import distance
ux = np.random.random_integers(0, L, 5000)
uy = np.random.random_integers(0, L, 5000)
upos = np.array([ux, uy]).T
hist = np.histogram(distance.pdist(upos), bins=150, normed=True)
ur = hist[1][1:] / diameter
uP = hist[0]

### Load simulated histogram

During simulation, the MC program samples the distribution of distances observed and saves it to the file `hist.dat`.
This we here load, normalize and plot.

In [ ]:
r,P = np.loadtxt('hist.dat', unpack=True)
r = r/diameter
P = P/P.cumsum()[-1] # normalize
plt.plot(r, P)
plt.plot(ur, uP)

### Calculate final distribution function, $g(r)$

In order to divide the two above distributions with each other, the data points in each set need to be aligned. This we easily ensure with Akima splines.

In [ ]:
from scipy.interpolate import Akima1DInterpolator
rvals = np.linspace(0,L/diameter,500)
gofr = Akima1DInterpolator(r, P)(rvals) / Akima1DInterpolator(ur, uP)(rvals)
plt.plot(rvals, gofr)
#plt.xlim((0,L/2/diameter))

### Exercises

0. For all figures, put labels on the axis as well as units
0. Explore the particle displacement parameter. How does this influence the mean square displacement and move acceptance? Does it influence the final result?
0. How does temperature enter the simulation?
0. Explain why $g(r)$ deviated from unity at large separations
0. Simulate $g(r)$ for non-interacting particles. Although it in theory should be, the curve may not be unity for all distances. Why?
0. List the differences between the particle simulator and the MC system